&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE.ipynb)

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;
[1]
[2](02-Intro_to_cuDF_UDFs.ipynb)
[3](03-Cudf_Exercise.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](02-Intro_to_cuDF_UDFs.ipynb)


Intro to cuDF
=======================

Welcome to first cuDF tutorial notebook! This is a short introduction to cuDF, partly modeled after 10 Minutes to Pandas, geared primarily for new users. cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating data. While we'll only cover some of the cuDF functionality, but at the end of this tutorial we hope you'll feel confident creating and analyzing GPU DataFrames. The tutorial is split into different modules as shown in the list below, based on their individual content and also contain embedded exercises for your practice.

For more information about CuDF refer to the documentation here: https://docs.rapids.ai/api/cudf/stable/
 
We'll start by introducing the pandas library, and quickly move on cuDF.

## Here is the list of contents in the lab:
- <a href='#pandas'>Pandas</a><br> A short introduction to the Pandas library, used commonly in data science for data manipulation and convenient storage.
- <a href='#dataframe'>CuDF Dataframes</a><br> This module shows you how to work with CuDF dataframes, the GPU equivalent of Pandas dataframes, for faster data transactions. It includes creating CuDF objects, viewing data, selecting data, boolean indexing and dealing with missing data.
- <a href='#operation'>Operations</a><br> Learn how to view descriptive statistics, perform string operations, concatenate, joins, append, group data and use applymap.
- <a href='#time'>TimeSeries</a><br> Introduction to using TimeSeries data format in CuDF   
- <a href='#condatarep'>Converting Data Representations</a><br> Here we will work with converting data representations, including CuPy, Pandas and Numpy, that are commonly required in data science pipelines.
- <a href='#datainout'>Getting Data In and Out</a><br> Transfering CuDf dataframes to and from CSV files.
- <a href='#performance'>Performance</a><br> Study the performance metrics of code written using CuDF compared to the regular data science operations and observe the benefits.
- <a href='#sensorex'>Sensor Data Example</a><br> Apply the fundamental methods of CuDF to work with a sample of sensor data.

In [55]:
import os
import numpy as np
import math
np.random.seed(12)

<a id="pandas"></a>
## Pandas

Data scientists typically work with two types of data: unstructured and structured. Unstructured data often comes in the form of text, images, or videos. Structured data - as the name suggests - comes in a structured form, often represented by a table or CSV. We'll focus the majority of these tutorials on working with structured data.

There exist many tools in the Python ecosystem for working with structured, tabular data but few are as widely used as Pandas. Pandas represents data in a table and allows a data scientist to manipulate the data to perform a number of useful operations such as filtering, transforming, aggregating, merging, visualizing and many more. 

For more information on Pandas, check out the excellent documentation: http://pandas.pydata.org/pandas-docs/stable/

Below we show how to create a Pandas DataFrame, an internal object for representing tabular data.

In [56]:
import pandas as pd; print('Pandas Version:', pd.__version__)


# here we create a Pandas DataFrame with
# two columns named "key" and "value"
df = pd.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

Pandas Version: 1.2.4
   key  value
0    0   10.0
1    0   11.0
2    2   12.0
3    2   13.0
4    3   14.0


We can perform many operations on this data. For example, let's say we wanted to sum all values in the in the `value` column. We could accomplish this using the following syntax:

In [57]:
aggregation = df['value'].sum()
print(aggregation)

60.0


<a id="cudf"></a>
## cuDF

Pandas is fantastic for working with small datasets that fit into your system's memory and workflows that are not computationally intense. However, datasets are growing larger and data scientists are working with increasingly complex workloads - the need for accelerated computing is increasing rapidly.

cuDF is a package within the RAPIDS ecosystem that allows data scientists to easily migrate their existing Pandas workflows from CPU to GPU, where computations can leverage the immense parallelization that GPUs provide.

Below, we show how to create a cuDF DataFrame.

In [4]:
import cudf; print('cuDF Version:', cudf.__version__)


# here we create a cuDF DataFrame with
# two columns named "key" and "value"
df = cudf.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
df

cuDF Version: 0.19.2


,key,value
0,0,10.0
1,0,11.0
2,2,12.0
3,2,13.0
4,3,14.0


As before, we can take this cuDF DataFrame and perform a `sum` operation over the `value` column. The key difference is that any operations we perform using cuDF use the GPU instead of the CPU.

In [5]:
aggregation = df['value'].sum()
print(aggregation)

60.0


Note how the syntax for both creating and manipulating a cuDF DataFrame is identical to the syntax necessary to create and manipulate Pandas DataFrames; the cuDF API is based on the Pandas API. This design choice minimizes the cognitive burden of switching from a CPU based workflow to a GPU based workflow and allows data scientists to focus on solving problems while benefitting from the speed of a GPU!

<a id='dataframe'></a>
# DataFrame Basics with cuDF

In the following tutorial, you'll get a chance to familiarize yourself with cuDF. For those of you with experience using pandas, this should look nearly identical.

Along the way you'll notice small exercises. These exercises are designed to help you get a feel for writing the code yourself, but if you get stuck, you can take a look at the solutions.

Portions of this were borrowed from the 10 Minutes to cuDF guide.

<a id="objcreation"></a>

Object Creation
---------------

Creating a `cudf.Series`.

In [6]:
s = cudf.Series([1,2,3,None,4])
print(s)

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64


Creating a `cudf.DataFrame` by specifying values for each column.

In [7]:
df = cudf.DataFrame({'a': list(range(20)),
'b': list(reversed(range(20))),
'c': list(range(20))})
print(df)

     a   b   c
0    0  19   0
1    1  18   1
2    2  17   2
3    3  16   3
4    4  15   4
5    5  14   5
6    6  13   6
7    7  12   7
8    8  11   8
9    9  10   9
10  10   9  10
11  11   8  11
12  12   7  12
13  13   6  13
14  14   5  14
15  15   4  15
16  16   3  16
17  17   2  17
18  18   1  18
19  19   0  19


Creating a `cudf.DataFrame` from a `pd.Dataframe`.

In [8]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
gdf = cudf.DataFrame.from_pandas(pdf)
print(gdf)

   a     b
0  0   0.1
1  1   0.2
2  2  <NA>
3  3   0.3


<a id='viewing'></a>

Viewing Data
-------------

Viewing the top rows of a GPU dataframe.

In [9]:
print(df.head(2))

   a   b  c
0  0  19  0
1  1  18  1


Sorting by values.

In [10]:
print(df.sort_values(by='b'))

     a   b   c
19  19   0  19
18  18   1  18
17  17   2  17
16  16   3  16
15  15   4  15
14  14   5  14
13  13   6  13
12  12   7  12
11  11   8  11
10  10   9  10
9    9  10   9
8    8  11   8
7    7  12   7
6    6  13   6
5    5  14   5
4    4  15   4
3    3  16   3
2    2  17   2
1    1  18   1
0    0  19   0


<a id='selection'></a>


Selection
------------

## Getting data

Selecting a single column, which initially yields a `cudf.Series` (equivalent to `df.a`).

In [11]:
print(df['a'])

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: a, dtype: int64


<a id='sellabel'></a>
## Selection by Label

Selecting rows from index 2 to index 5 from columns `a` and `b`.

In [12]:
print(df.loc[2:5, ['a', 'b']])

   a   b
2  2  17
3  3  16
4  4  15
5  5  14


<a id='selpos'></a>

## Selection by Position

Selecting via integers and integer slices, like numpy/pandas.

In [13]:
df.iloc[0]

a     0
b    19
c     0
Name: 0, dtype: int64

In [14]:
df.iloc[0:3, 0:2]

,a,b
0,0,19
1,1,18
2,2,17


You can also select elements of a `DataFrame` or `Series` with direct index access.

In [15]:
df[3:5]

,a,b,c
3,3,16,3
4,4,15,4


In [16]:
s[3:5]

3    <NA>
4       4
dtype: int64

<a id='exercise1'></a>

## Exercise 1

Try to select only the rows at index `4` and `9` from `df`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.iloc[[4,9]])
   </pre>
</details>

<a id='boolean'> </a>

## Boolean Indexing

Selecting rows in a `DataFrame` or `Series` by direct Boolean indexing.

In [17]:
print(df[df.b > 15])

   a   b  c
0  0  19  0
1  1  18  1
2  2  17  2
3  3  16  3


Selecting values from a `DataFrame` where a Boolean condition is met, via the `query` API.

In [18]:
print(df.query("b == 3 AND c <= 3 "))  

     a  b   c
16  16  3  16


In [19]:
val = 3
df.query("b == @val")

,a,b,c
16,16,3,16


You can also pass local variables to cuDF queries, via the `local_dict` keyword or `@` operator.

In [20]:
cudf_comparator = 3
print(df.query("b == @cudf_comparator"))

     a  b   c
16  16  3  16


Supported logical operators include `>`, `<`, `>=`, `<=`, `==`, and `!=`.

<a id='exercise2'></a>

## Exercise 2

Try to select only the rows from `df` where the value in column `b` is greater than the value in column `c` + 6.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.query("b > c + 6"))
   </pre>
</details>

<a id='missing'></a>

Missing Data
------------

Missing data can be replaced by using the `fillna` method.

In [21]:
print(s.fillna(999))

0      1
1      2
2      3
3    999
4      4
dtype: int64


Missing data can be dropped by using the `dropna` method.

In [22]:
print(s.dropna())

0    1
1    2
2    3
4    4
dtype: int64


<a id='operation'></a>

Operations
------------

<a id='stats'></a>

## Stats

Calculating descriptive statistics for a `Series`.

In [23]:
s = cudf.Series(np.arange(10)).astype(np.float32)

In [24]:
print(s.mean(), s.var(), s.std(), s.kurtosis())

4.5 9.166666666666668 3.0276503540974917 -1.200000000000001


In [25]:
print(df.describe())

              a         b         c
count  20.00000  20.00000  20.00000
mean    9.50000   9.50000   9.50000
std     5.91608   5.91608   5.91608
min     0.00000   0.00000   0.00000
25%     4.75000   4.75000   4.75000
50%     9.50000   9.50000   9.50000
75%    14.25000  14.25000  14.25000
max    19.00000  19.00000  19.00000


<a id='applymap'></a>

## Applymap

Applying functions to a `Series`.

In [26]:
print(s.applymap(lambda x: (x ** 2) + (x / 2)))

0     0.0
1     1.5
2     5.0
3    10.5
4    18.0
5    27.5
6    39.0
7    52.5
8    68.0
9    85.5
dtype: float64


In [27]:
def add_ten(num):
    return num + 10

print(s.applymap(add_ten))

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64


<a id='string'></a>

## String Methods

Like pandas, cuDF provides string processing methods in the `str` attribute of `Series`. Full documentation of string methods is a work in progress. Please see the [cuDF](https://docs.rapids.ai/api/cudf/nightly/) and [nvStrings](https://docs.rapids.ai/api/nvstrings/nightly/) API documentation for more information.

In [28]:
s = cudf.Series(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'])
print(s.str.lower())

0       a
1       b
2       c
3    aaba
4    baca
5    <NA>
6    caba
7     dog
8     cat
dtype: object


To check the number of bytes in each string of the series:

In [29]:
print(s.str.byte_count())

0       1
1       1
2       1
3       4
4       4
5    <NA>
6       4
7       3
8       3
dtype: int32


To find whether a word or regular expression occurs in the strings of the series:

In [30]:
print(s.str.contains('C|dog', regex=True))

0    False
1    False
2     True
3    False
4    False
5     <NA>
6     True
7     True
8    False
dtype: bool


<a id='exercise3'></a>

## Exercise 3

Try to convert all the strings to uppercase. Take a look at the nvStrings API documentation linked above if you need some help.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(s.str.upper())
   </pre>
</details>

<a id='concat'></a>

## Concat

Concatenating `Series` and `DataFrames` row-wise.

In [31]:
s = cudf.Series([1, 2, 3, None, 5])
print(cudf.concat([s, s]))

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64


<a id='append'></a>

## Append

Appending values from another `Series` or array-like object.

In [32]:
print(s.append(s))

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64


<a id='join'></a>

## Join

Performing SQL style merges. Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [33]:
df_a = cudf.DataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = cudf.DataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left')
print(merged)

  key  vals_a vals_b
0   a    10.0  100.0
1   c    12.0  101.0
2   e    14.0  102.0
3   b    11.0   <NA>
4   d    13.0   <NA>


<a id='exercise4'></a>

## Exercise 4

Using the DataFrames we created above, try to do an `inner` join using `merge`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df_a.merge(df_b, on=['key'], how='inner'))
   </pre>
</details>

<a id='grouping'></a>

## Grouping

Like pandas, cuDF supports the [Split-Apply-Combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) groupby paradigm.

In [34]:
df['agg_col1'] = [1 if x % 2 == 0 else 0 for x in range(len(df))]
df['agg_col2'] = [1 if x % 3 == 0 else 0 for x in range(len(df))]

Grouping and then applying the `sum` function to the grouped data.

In [35]:
df.groupby('agg_col1').sum()

,a,b,c,agg_col2
agg_col1,,,,
1,90,100,90,4
0,100,90,100,3


Grouping and applying statistical functions to specific columns, using `agg`.

In [36]:
df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'})

,a,b,c
agg_col1,,,
1,18,10.0,90
0,19,9.0,100


<a id='exercise5'></a>

## Exercise 5

We can also group by multiple columns at once, which we call grouping hierarchically. Try to group `df` by `agg_col1` and `agg_col2` and calculate the mean of column `a` and minimum of column `b`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>df.groupby(['agg_col1', 'agg_col2']).agg({'a':'mean', 'b':'min'})
   </pre>
</details>

<a id='time'></a>


Time Series
------------


`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [37]:
date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
date_df['value'] = np.random.sample(len(date_df))
print(date_df.head())

        date     value
0 2018-11-20  0.154163
1 2018-11-21  0.740050
2 2018-11-22  0.263315
3 2018-11-23  0.533739
4 2018-11-24  0.014575


<a id='exercise6'></a>

## Exercise 6

Try to use `query` to filter `date_df` to only those row with a date before `2018-11-23`. This is a bit trickier than the prior exercises. As a hint, you'll want to explore the `to_datetime` function from the `pandas` library.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>
    search_date = pd.to_datetime('2018-11-23')
    date_df.loc[date_df.date &lt= search_date]
            </br>
   </pre>
</details>

You can also interact with datetime columns to extract things like the day, hour, minute, and more.

In [38]:
date_df['minute'] = date_df.date.dt.minute
print(date_df.head())

        date     value  minute
0 2018-11-20  0.154163       0
1 2018-11-21  0.740050       0
2 2018-11-22  0.263315       0
3 2018-11-23  0.533739       0
4 2018-11-24  0.014575       0


<a id='condatarep'></a>

Converting Data Representation
--------------------------------

<a id='cupy'></a>

## CuPy

Combining cuDF and CuPy is particularly useful (it's the GPU equivalent of combining pandas and NumPy), so we've put together a Getting Started [guide](https://docs.rapids.ai/api/cudf/nightly/10min-cudf-cupy.html). Let's start of by importing cupy and doing data operations

In [39]:
import cupy as cp

First we will convert cuDF DataFrame to a CuPy ndarray, using DLpack which is the best method:

In [40]:
nelem = 10000
df = cudf.DataFrame({'a':range(nelem),
                     'b':range(500, nelem + 500),
                     'c':range(1000, nelem + 1000)}
                   )

%time arr_cupy = cp.fromDlpack(df.to_dlpack())
arr_cupy

CPU times: user 220 µs, sys: 114 µs, total: 334 µs
Wall time: 236 µs


array([[    0,   500,  1000],
       [    1,   501,  1001],
       [    2,   502,  1002],
       ...,
       [ 9997, 10497, 10997],
       [ 9998, 10498, 10998],
       [ 9999, 10499, 10999]])



For now, note that you can convert a DataFrame or a Series to a CuPy array with `.values`. 

In [41]:
df.values

array([[    0,   500,  1000],
       [    1,   501,  1001],
       [    2,   502,  1002],
       ...,
       [ 9997, 10497, 10997],
       [ 9998, 10498, 10998],
       [ 9999, 10499, 10999]])

<a id='pandas'></a>

## Pandas

Converting a cuDF `DataFrame` to a pandas `DataFrame`.

In [42]:
df.head().to_pandas()

,a,b,c
0,0,500,1000
1,1,501,1001
2,2,502,1002
3,3,503,1003
4,4,504,1004


<a id='numpy'></a>

## Numpy

Converting a cuDF `DataFrame` to a numpy `ndarray`.

In [43]:
df.as_matrix()

array([[    0,   500,  1000],
       [    1,   501,  1001],
       [    2,   502,  1002],
       ...,
       [ 9997, 10497, 10997],
       [ 9998, 10498, 10998],
       [ 9999, 10499, 10999]])

Converting a cuDF `Series` to a numpy `ndarray`.

In [44]:
print(cp.asarray(df['a']))

[   0    1    2 ... 9997 9998 9999]


<a id='datainout'></a>

Getting Data In/Out
------------------------


<a id='csv'></a>

## CSV

Writing to a CSV file, using a GPU-accelerated CSV writer.

In [58]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')
    
df.to_csv('example_output/foo.csv', index=False)

Reading from a csv file.

In [46]:
df = cudf.read_csv('example_output/foo.csv')
print(df)

         a      b      c
0        0    500   1000
1        1    501   1001
2        2    502   1002
3        3    503   1003
4        4    504   1004
...    ...    ...    ...
9995  9995  10495  10995
9996  9996  10496  10996
9997  9997  10497  10997
9998  9998  10498  10998
9999  9999  10499  10999

[10000 rows x 3 columns]


That's it! You've got the basics of cuDF down! Let's talk a little bit about the computational performance of cuDF and GPUs.

<a id='performance'></a>


# Performance

One of the primary reasons to use cuDF over pandas is performance. For some workflows, the GPU can be **much** faster than the CPU. Let's illustrate this by starting with a small example: creating a DataFrame and calculating the sum of a column.

In [47]:
a = np.random.rand(10000000) # 10 million values

In [48]:
pdf = pd.DataFrame({'a': a})
cdf = cudf.DataFrame({'a': a})

In [49]:
%%timeit
pdf['a'].sum()

13.9 ms ± 39.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
%%timeit
cdf['a'].sum()

1.28 ms ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Pretty cool! This is a pretty small example, though.

<a id='sensorex'></a>

### A More Realistic Example: Sensor Data Analytics

To get a more realistic sense of how powerful cuDF and GPUs can be, let's imagine you had a fleet of sensors that collect data every millisecond. These sensors could be measuring pressure, temperature, or something else entirely.

Let's imagine we want to analyze one day's worth of sensor data. We'll assign random values for the sensor `value` to use for this example. We'll start by creating the data, and generating some datetime related features like we learned about in the above tutorial.

In [51]:
%%time

date_df = pd.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute

print(date_df.shape)
date_df.head()

(86400001, 4)
CPU times: user 22.7 s, sys: 3.56 s, total: 26.2 s
Wall time: 26.2 s


,date,value,hour,minute
0,2019-07-05 00:00:00.000,0.391385,0,0
1,2019-07-05 00:00:00.001,0.722956,0,0
2,2019-07-05 00:00:00.002,0.062759,0,0
3,2019-07-05 00:00:00.003,0.723346,0,0
4,2019-07-05 00:00:00.004,0.914508,0,0


Just creating the data takes a while! Let's do our analysis. From our sensor data, we want to get the maximum sensor value for each minute. Since we don't want to combine values in the same minute of different hours, we'll need to do a hierarchical groupby.

In [52]:
%time results = date_df.groupby(['hour', 'minute']).agg({'value':'max'})
results.head()

CPU times: user 2.79 s, sys: 1.2 s, total: 3.99 s
Wall time: 3.99 s


value
hour minute          
0    0       0.999994
     1       0.999931
     2       0.999995
     3       0.999962
     4       0.999975

This is fairly slow! Imagine if we had a fleet of sensors. Time would become a serious concern.

Let's try this in cuDF now, using the GPU DataFrame. We'll run the same code as above.

In [53]:
%%time

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute
date_df['second'] = date_df.date.dt.second

print(date_df.shape)
print(date_df.head())

(86400001, 5)
                     date     value  hour  minute  second
0 2019-07-05 00:00:00.000  0.453388     0       0       0
1 2019-07-05 00:00:00.001  0.804337     0       0       0
2 2019-07-05 00:00:00.002  0.449341     0       0       0
3 2019-07-05 00:00:00.003  0.596637     0       0       0
4 2019-07-05 00:00:00.004  0.552345     0       0       0
CPU times: user 1.75 s, sys: 745 ms, total: 2.49 s
Wall time: 2.49 s


In [54]:
%time results = date_df.groupby(['hour', 'minute', 'second']).agg({'value':'max'})
results.head()

CPU times: user 14.5 ms, sys: 20.8 ms, total: 35.3 ms
Wall time: 34.2 ms


,,,value
hour,minute,second,
2,43,35,0.999544
4,13,8,0.997685
1,44,5,0.999127
3,31,6,0.998985
9,49,13,0.994060


The data creation accelerated from 23 s to 2.5 seconds and the processing from 3 s to 40 ms.

# Conclusion

Now we are familiar with creating CUDF dataframes, selecting, viewing and manipulating data. The operations are almost the same as pandas, and can easily replace the pandas operations in our traditional data science pipeline. While the results may vary slightly on different GPUs, it should be clear that GPU acceleration can make a significant difference. We can get much faster results with the same code! The next tutorial will give us more control over the data and allow us to play with user defined functions. The kernel function allow us to specify how the data is chunked, thus we can accelerate the row-wise operations with greater control over execution.

## Licensing
  
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).

If you want to learn about implementing machine learning algorithms in CuML using CuPy as the data input format, refer to the bonus lab here.

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&ensp;
[1]
[2](02-Intro_to_cuDF_UDFs.ipynb)
[3](03-Cudf_Exercise.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](02-Intro_to_cuDF_UDFs.ipynb)


&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;
[Home Page](../START_HERE.ipynb)